# Sample QC

This notebook is a quick guide to how sketching algorithms can be used for sample QC. The particular emphasis is on getting from raw sequencing data to filtered reads, which we will then use in the next steps of our genomics workflow.

> Note: many sketching algorithms can work with raw data, which contributes to the speed benefits of sketching. That being said, if we want more than just a quick and dirty analysis, it's always best to QC your data...

We'll use the following sketching algorithms, implemented via some excellent open-source bioinformatics software:

* MinHash
* CountMin sketch
* Bloom filter

***

## The data

As mentioned in 4.1, we are looking at a suspected outbreak of *E.cloacae* in a hospital. We are using the sequence data collected during the [Reuter et al.](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4001082/) study. The following table contains the accessions for the isolates we need, I just grabbed this from the [supplementary file](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4001082/bin/NIHMS58061-supplement-Supplementary_Online_Content.pdf) and then added the ENA experiment and run IDs.

|isolate name|patient ID|ENA biosample|ENA experiment|ENA run|
|-|-|-|-|-|
EC1a|EC1|ERS184249|ERX168346|ERR193657|
EC2a|EC2|ERS184250|ERX168347|ERR193658|
EC2b|EC2|ERS184251|ERX168341|ERR193652|
EC3a|EC3|ERS184252|ERX168340|ERR193651|
EC4a|EC4|ERS184245|ERX168345|ERR193656|
EC5a|EC5|ERS184246|ERX168339|ERR193650|
EC6a|EC6|ERS184247|ERX168343|ERR193654|
EC7a|EC7|ERS184248|ERX168344|ERR193655|

These samples correspond to whole genome sequence data for *E.cloacae* isolates, collected from several patients in the hospital during the suspected outbreak.

To save us the pain of waiting for fastq-dump to collect the data, I have already downloaded it. For future reference though, use `fastq-dump` (part of [sra-tools](https://github.com/ncbi/sra-tools)) to download this data from the ENA. We could have sketched this download data stream, but as we know that we need this data for our analysis it is best to have it stored on disk. You'll see later how we can sketch a data stream from a download, evaluate these sketches to see if the data is helpful for our analysis, and then decide to store or discard it.

The sequence data for these isolates is actually pretty poor quality - [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) showed the terminal 50 bases to be full of Ns and the quality dropped below Q20. So I ran Trimmomatic to clean the data, here is the command ran for one sample:

```
trimmomatic PE ERX168346_1.fastq ERX168346_2.fastq ERX168346_1-trimmed.fq ERX168346_1-se ERX168346_2-trimmed.fq ERX168346_2-se SLIDINGWINDOW:5:20 MINLEN:100
```

So, we now have quality checked, trimmed sequence data for each sample downloaded and ready to go. This is stored in `../data/reads`. To make things easier to follow, the following steps will be ran on just one of these samples. Feel free to update the code to run on all of them!

## Sanity check

Are my samples what I think they are?!

The study labelled these isolates as *E.cloacae*, let's use **MinHash** to check. We'll be using the excellent [sourmash](https://github.com/dib-lab/sourmash) software. The sourmash docs actually have loads of great examples and workflows - be sure to check them out too.


* start with downloading a reference database of MinHash genome sketches:

In [ ]:
# download the pre-made MinHash database (courtesy of sourmash) containing all GenBank microbial genomes
!wget -O genbank-k31.lca.json.gz https://osf.io/4f8n3/download

* next, create a MinHash sketch of the reads from one sample:

In [ ]:
# sketch the reads from a sample
!sourmash compute --scaled 1000 -k 11,21,31 ../data/reads/ERX168346_*-trimmed.fq.gz --merge ERX168346 -o ERX168346.sketch 

> the --scaled flag tells sourmash to decide the number of hashes to include in the MinHash sketch. Sourmash decides this based on the sequence length; it is effectively setting a compression ratio of 1000-to-1.

> we have given 3 k-mer sizes with the `--k` flag, so will get 3 sketches

> the --merge flag is to tell sourmash that we have 2 read files (paired end data). We will still get 3 sketches but each one will be made from both read files.

* now we have sketched our sample, we can compare it to the reference sketches and check we have the organism we expect (e.cloacae!):

In [ ]:
# compare the sketch of the reads against each sketch in the reference database
!sourmash gather ERX168346.sketch genbank-k31.lca.json.gz

> note: the above command is sometimes killed by Binder as memory limits are exceed when the database is loaded. We can try an alternative approach that uses less memory:

* perform a direct comparison between sample and the *e.cloacae* reference genome:

In [ ]:
!sourmash compute -k 11 --scaled 1000  ../data/GCF_000025565.1_ASM2556v1_genomic.fna.gz -o genome.sketch
!sourmash search --containment genome.sketch ERX168346.sketch

So the sketch from our sample matches the sketch of the *E.cloacae* reference genome - yay!

It wasn't a complete match however, there is a little bit of a difference between the sketches. This is probably due to sequencing error. We will have a fair few unique k-mers in the sequencing data, which typically arise from errors during sequencing (but could also be from strain variation).

Let's use some more sketching to investigate!


## K-mer spectrum

First off, let's check that our guess is right. Do we have a lot of unique k-mers? To answer this, we'll use the **Count-Min sketch** and another piece of software from Titus Brown et al. - [khmer](https://github.com/dib-lab/khmer).

* begin by using a Count-Min sketch to create a k-mer count graph (the k-mer spectrum):

In [ ]:
!abundance-dist-single.py -M 1e9 -k 21 -s ../data/reads/ERX168346_1-trimmed.fq.gz ERX168346.dist

* plot the k-mer abundance against k-mer count:

In [ ]:
%matplotlib inline
import numpy
from pylab import *
dist1 = numpy.loadtxt('ERX168346.dist', skiprows=1, delimiter=',')
plot(dist1[:,0], dist1[:,1])
axis(xmax=200)
title('k-mer abundance histogram')
xlabel('k-mer abundance')
ylabel('N k-mers with that abundance')

We can see a a lot unique k-mers in that plot. One reason to remove these is to reduce the amount of data we process in downstream analysis, which will reduce analysis time and take less memory (as there are fewer unique k-mers to hold in memory). That being said, it's not really necessary and removing them can have a negative impact on some analyses as we would be reducing coverage.

So instead of removing them, let's first try to correct the sequence reads.

## Read error correction using Bloom filters

For this, we can use [lighter](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-014-0509-9). We need a bit of information first:

* what size is the genome we have sequenced
* how many reads do we have

This information is used to calculate the sequencing coverage, this is then used to set the sampling fraction (alpha) so that it is inversely proportion to the depth of sequencing.

We saw in the output of our earlier sourmash command that we have 574469 reads per file (read length ~140bp) for ERX168346. We also know that the genome size of *E.cloacae* is 5.31Mb. So our genome coverage from this sample is approximately 7X (which isn't great, but we did aggressively trim the data as the quality was poor).

We calculate our sampling fraction for each file using the equation from the lighter paper: `7/coverage`.

This means our sampling fraction is 2.13

* now let's run lighter:

In [ ]:
!lighter -r ../data/reads/ERX168346_1-trimmed.fq.gz -k 17 5310000 2.13
!lighter -r ../data/reads/ERX168346_2-trimmed.fq.gz -k 17 5310000 2.13

> lighter needs a k-mer size, the genome size and the alpha. Alpha is optional, but if you don't include it lighter will do an extra pass of your data to calculate it for you.

Lighter corrected around 36000 bases - great! Let's now do some low-abundance k-mer trimming.


## K-mer trimming using Count-Min sketch

Back to the Count-Min sketch and khmer...

* perform k-mer frequency based trimming on the corrected reads from lighter:

> this takes a little while

In [ ]:
!trim-low-abund.py -V -M 1e9 -C 3 -Z 10 -o ERX168346.corrected.trimmed.fq ERX168346_1-trimmed.cor.fq.gz ERX168346_2-trimmed.cor.fq.gz


* let's plot the k-mer spectrum again now the reads have been corrected and k-mer trimmed:

In [ ]:
!abundance-dist-single.py -M 1e9 -k 21 -s ERX168346.corrected.trimmed.fq ERX168346.dist

dist1 = numpy.loadtxt('ERX168346.dist', skiprows=1, delimiter=',')
plot(dist1[:,0], dist1[:,1])
axis(xmax=200)
title('k-mer abundance histogram')
xlabel('k-mer abundance')
ylabel('N k-mers with that abundance')

So we've managed to reduce the number of low abundance k-mers in our sample using sketching.

***

Let's move on to the next stage of the workflow: [resistome profiling](r4.3.Resistome-profiling.ipynb)